<a href="https://colab.research.google.com/github/ywChen-NTUST/DS_hw3/blob/main/DS_hw3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download dataset into workspace

In [ ]:
!gdown --id "1PsnRV_D0ZJ6p6dolK4wX87hYk1qZ2UkX" --output "data.csv"
!gdown --id "1pu2RRUgOaEPB92Pq5G-zVcBwglQOEKSm" --output "prediction.csv"

Downloading...
From: https://drive.google.com/uc?id=1PsnRV_D0ZJ6p6dolK4wX87hYk1qZ2UkX
To: /content/data.csv
45.6MB [00:00, 110MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1pu2RRUgOaEPB92Pq5G-zVcBwglQOEKSm
To: /content/prediction.csv
100% 268k/268k [00:00<00:00, 36.6MB/s]


In [ ]:
!ls

data.csv  prediction.csv  sample_data


# Data cleaning

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv("data.csv")
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010/12/1 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010/12/1 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010/12/1 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010/12/1 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010/12/1 08:26,3.39,17850.0,United Kingdom


In [ ]:
data_uk = data[data["Country"]=="United Kingdom"]
data_uk.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010/12/1 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010/12/1 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010/12/1 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010/12/1 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010/12/1 08:26,3.39,17850.0,United Kingdom


In [ ]:
na = pd.isna(data_uk["InvoiceNo"])
na.to_list().count(True)

0

In [ ]:
cancelIndex = data_uk[data_uk["InvoiceNo"].str.startswith("C")].index
data_uk_cancel = data_uk.drop(cancelIndex)
data_uk_cancel.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010/12/1 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010/12/1 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010/12/1 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010/12/1 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010/12/1 08:26,3.39,17850.0,United Kingdom


In [ ]:
postageIndex = data_uk_cancel[data_uk_cancel["Description"] == "POSTAGE"].index
data_uk_cancel_postage = data_uk_cancel.drop(postageIndex)
data_uk_cancel_postage.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010/12/1 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010/12/1 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010/12/1 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010/12/1 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010/12/1 08:26,3.39,17850.0,United Kingdom


In [ ]:
data.shape, data_uk.shape, data_uk_cancel.shape, data_uk_cancel_postage.shape

((541909, 8), (495478, 8), (487622, 8), (487570, 8))

# Convert format

In [ ]:
from mlxtend.preprocessing import TransactionEncoder

In [ ]:
txDataDict = {}
nan = 0

for label, row in data_uk_cancel_postage.iterrows():
  if row["Description"] == np.nan:
    nan += 1
  elif row["InvoiceNo"] in txDataDict:
    txDataDict[row["InvoiceNo"]].append(str(row["Description"]))
  else:
    txDataDict[row["InvoiceNo"]] = [str(row["Description"])]

txDataDict["538504"], nan

(['GREEN WIRE STANDING CANDLE HOLDER',
  'NATURAL SLATE HEART CHALKBOARD ',
  'ANTIQUE SILVER TEA GLASS ETCHED',
  'AGED GLASS SILVER T-LIGHT HOLDER',
  'FRENCH BLUE METAL DOOR SIGN 6',
  'FRENCH BLUE METAL DOOR SIGN 5',
  'FRENCH BLUE METAL DOOR SIGN 2',
  'ROTATING LEAVES T-LIGHT HOLDER',
  'ANTIQUE SILVER TEA GLASS ETCHED',
  'CHRISTMAS GINGHAM HEART',
  'WOODEN SCHOOL COLOURING SET',
  'ANTIQUE GLASS HEART DECORATION ',
  'ROTATING LEAVES T-LIGHT HOLDER',
  'HOT WATER BOTTLE BABUSHKA ',
  'CARDHOLDER HOLLY WREATH METAL',
  'WICKER WREATH LARGE',
  'HEART OF WICKER SMALL',
  'WICKER STAR '],
 0)

In [ ]:
txData = list(txDataDict.values())
txData[:5]

[['WHITE HANGING HEART T-LIGHT HOLDER',
  'WHITE METAL LANTERN',
  'CREAM CUPID HEARTS COAT HANGER',
  'KNITTED UNION FLAG HOT WATER BOTTLE',
  'RED WOOLLY HOTTIE WHITE HEART.',
  'SET 7 BABUSHKA NESTING BOXES',
  'GLASS STAR FROSTED T-LIGHT HOLDER'],
 ['HAND WARMER UNION JACK', 'HAND WARMER RED POLKA DOT'],
 ['ASSORTED COLOUR BIRD ORNAMENT',
  "POPPY'S PLAYHOUSE BEDROOM ",
  "POPPY'S PLAYHOUSE KITCHEN",
  'FELTCRAFT PRINCESS CHARLOTTE DOLL',
  'IVORY KNITTED MUG COSY ',
  'BOX OF 6 ASSORTED COLOUR TEASPOONS',
  'BOX OF VINTAGE JIGSAW BLOCKS ',
  'BOX OF VINTAGE ALPHABET BLOCKS',
  'HOME BUILDING BLOCK WORD',
  'LOVE BUILDING BLOCK WORD',
  'RECIPE BOX WITH METAL HEART',
  'DOORMAT NEW ENGLAND'],
 ['JAM MAKING SET WITH JARS',
  'RED COAT RACK PARIS FASHION',
  'YELLOW COAT RACK PARIS FASHION',
  'BLUE COAT RACK PARIS FASHION'],
 ['BATH BUILDING BLOCK WORD']]

In [ ]:
encoder = TransactionEncoder()
encode = encoder.fit(txData).transform(txData)
txDataPD = pd.DataFrame(encode, columns=encoder.columns_)
txDataPD.head()

,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,TOADSTOOL BEDSIDE LIGHT,TRELLIS COAT RACK,*Boombox Ipod Classic,*USB Office Mirror Ball,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 HANGING EGGS HAND PAINTED,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE SKULLS,12 PENCILS TALL TUBE WOODLAND,12 PINK HEN+CHICKS IN BASKET,12 PINK ROSE PEG PLACE SETTINGS,12 RED ROSE PEG PLACE SETTINGS,15 PINK FLUFFY CHICKS IN BOX,15CM CHRISTMAS GLASS BALL 20 LIGHTS,16 PC CUTLERY SET PANTRY DESIGN,16 PIECE CUTLERY SET PANTRY DESIGN,18PC WOODEN CUTLERY SET DISPOSABLE,2 DAISIES HAIR COMB,2 PICTURE BOOK EGGS EASTER BUNNY,2 PICTURE BOOK EGGS EASTER CHICKS,2 PICTURE BOOK EGGS EASTER DUCKS,20 DOLLY PEGS RETROSPOT,...,sold as set on dotcom,sold as set on dotcom and amazon,sold as set/6 by dotcom,sold in set?,sold with wrong barcode,stock check,stock creditted wrongly,taig adjust,taig adjust no stock,temp adjustment,test,throw away,thrown away,thrown away-can't sell,thrown away-can't sell.,to push order througha s stock was,water damage,water damaged,website fixed,wet,wet boxes,wet damaged,wet pallet,wet rusty,wet/rusty,wet?,wrong barcode,wrong barcode (22467),wrong code,wrong code?,wrongly coded 20713,wrongly coded 23343,wrongly coded-23343,wrongly marked,wrongly marked 23343,wrongly marked carton 22804,wrongly marked. 23343 in box,wrongly sold (22719) barcode,wrongly sold as sets,wrongly sold sets
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False

# Association rule

In [ ]:
from mlxtend.frequent_patterns import apriori, association_rules

In [ ]:
frequent_itemsets = apriori(txDataPD, min_support=0.01, use_colnames=True)

In [ ]:
result = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)

# Apply rule

In [ ]:
prediction = pd.read_csv("prediction.csv")
prediction.head()

,index,Association Rule antecedents,Association Rule consequents
0,0,BLUE SPOT CERAMIC DRAWER KNOB,WHITE SPOT BLUE CERAMIC DRAWER KNOB
1,1,Lighthouse Trading zero invc incorr,"SILVER GLASS T-LIGHT SET, RED RETROSPOT UMBRELLA"
2,2,"LUNCH BAG SUKI DESIGN, LUNCH BAG APPLE DESIGN",LUNCH BAG SPACEBOY DESIGN
3,3,ELEPHANT BIRTHDAY CARD,SILVER ROCCOCO CHANDELIER
4,4,JUMBO BAG PINK VINTAGE PAISLEY,JUMBO BAG RED RETROSPOT


In [ ]:
labels = []

for label, row in prediction.iterrows():
  antecedent = set(row["Association Rule antecedents"].split(", "))
  consequent = set(row["Association Rule consequents"].split(", "))
  
  possibleCons = result[result["antecedents"] == antecedent]["consequents"]

  lab = 0
  for cons in possibleCons:
    if (cons == consequent):
      lab = 1

  labels.append(lab)

In [ ]:
len(labels)

3055

# Download result

In [ ]:
with open("result.csv", 'w') as fh:
  fh.write("index,label"+"\n")
  for i in range(len(labels)):
    fh.write(str(i)+","+str(int(labels[i]))+"\n")

In [ ]:
from google.colab import files
files.download('result.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Observation

In [ ]:
max_confidence = result[["antecedents", "consequents", "antecedent support", "consequent support", "support", "confidence"]].sort_values(by=['confidence', "support"], ascending=False)
max_confidence

,antecedents,consequents,antecedent support,consequent support,support,confidence
16,(BEADED CRYSTAL HEART PINK ON STICK),(DOTCOM POSTAGE),0.010258,0.035255,0.010009,0.975728
276,"(SUKI SHOULDER BAG, JAM MAKING SET PRINTED)",(DOTCOM POSTAGE),0.010806,0.035255,0.010407,0.963134
602,"(REGENCY TEA PLATE PINK, REGENCY TEA PLATE ROS...",(REGENCY TEA PLATE GREEN ),0.011752,0.016731,0.011104,0.944915
608,"(WOODEN TREE CHRISTMAS SCANDINAVIAN, WOODEN HE...",(WOODEN STAR CHRISTMAS SCANDINAVIAN),0.011553,0.023852,0.010905,0.943966
106,(HERB MARKER THYME),(HERB MARKER ROSEMARY),0.011453,0.011503,0.010656,0.930435
...,...,...,...,...,...,...
107,(LOVE HOT WATER BOTTLE),(HOT WATER BOTTLE KEEP CALM),0.022657,0.037596,0.011353,0.501099
618,"(CHARLOTTE BAG SUKI DESIGN, RED RETROSPOT CHAR...","(STRAWBERRY CHARLOTTE BAG, CHARLOTTE BAG PINK ...",0.023653,0.018026,0.011851,0.501053
629,"(RED RETROSPOT CHARLOTTE BAG, CHARLOTTE BAG PI...","(CHARLOTTE BAG SUKI DESIGN, WOODLAND CHARLOTTE...",0.023852,0.020566,0.011951,0.501044
559,"(LUNCH BAG CARS BLUE, LUNCH BAG RED RETROSPOT)",(LUNCH BAG WOODLAND),0.025346,0.043372,0.012698,0.500982


In [ ]:
result[["antecedents", "consequents", "antecedent support", "consequent support", "support", "confidence"]].sort_values(by=['support', 'confidence'], ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence
118,(JUMBO BAG PINK POLKADOT),(JUMBO BAG RED RETROSPOT),0.057713,0.096504,0.039090,0.677308
78,(GREEN REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER ),0.046509,0.047655,0.034907,0.750535
79,(ROSES REGENCY TEACUP AND SAUCER ),(GREEN REGENCY TEACUP AND SAUCER),0.047655,0.046509,0.034907,0.732497
132,(JUMBO STORAGE BAG SUKI),(JUMBO BAG RED RETROSPOT),0.056269,0.096504,0.034757,0.617699
130,(JUMBO SHOPPER VINTAGE RED PAISLEY),(JUMBO BAG RED RETROSPOT),0.056419,0.096504,0.032716,0.579876
...,...,...,...,...,...,...
315,"(JUMBO BAG WOODLAND ANIMALS, DOTCOM POSTAGE)",(RECYCLING BAG RETROSPOT ),0.017976,0.035355,0.010009,0.556787
664,"(JUMBO SHOPPER VINTAGE RED PAISLEY, DOTCOM POS...","(JUMBO BAG RED RETROSPOT, JUMBO STORAGE BAG SUKI)",0.018076,0.034757,0.010009,0.553719
376,"(JUMBO BAG RED RETROSPOT, JUMBO BAG OWLS)",(JUMBO BAG PINK POLKADOT),0.018424,0.057713,0.010009,0.543243
661,"(JUMBO SHOPPER VINTAGE RED PAISLEY, JUMBO BAG ...",(DOTCOM POSTAGE),0.019122,0.035255,0.010009,0.523438


In [ ]:
result[["antecedents", "consequents", "antecedent support", "consequent support", "support", "confidence"]].sort_values(by=['antecedent support', 'confidence'], ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence
118,(JUMBO BAG PINK POLKADOT),(JUMBO BAG RED RETROSPOT),0.057713,0.096504,0.039090,0.677308
130,(JUMBO SHOPPER VINTAGE RED PAISLEY),(JUMBO BAG RED RETROSPOT),0.056419,0.096504,0.032716,0.579876
132,(JUMBO STORAGE BAG SUKI),(JUMBO BAG RED RETROSPOT),0.056269,0.096504,0.034757,0.617699
154,(LUNCH BAG SUKI DESIGN ),(LUNCH BAG RED RETROSPOT),0.050891,0.069316,0.025695,0.504892
153,(LUNCH BAG PINK POLKADOT),(LUNCH BAG RED RETROSPOT),0.050095,0.069316,0.027587,0.550696
...,...,...,...,...,...,...
103,(HERB MARKER PARSLEY),(HERB MARKER THYME),0.011353,0.011453,0.010308,0.907895
97,(HERB MARKER PARSLEY),(HERB MARKER MINT),0.011353,0.011602,0.010208,0.899123
91,(HERB MARKER PARSLEY),(HERB MARKER BASIL),0.011353,0.011553,0.010109,0.890351
276,"(SUKI SHOULDER BAG, JAM MAKING SET PRINTED)",(DOTCOM POSTAGE),0.010806,0.035255,0.010407,0.963134


In [ ]:
result[["antecedents", "consequents", "antecedent support", "consequent support", "support", "confidence"]].sort_values(by=['consequent support', 'confidence'], ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence
22,(CANDLEHOLDER PINK HANGING HEART),(WHITE HANGING HEART T-LIGHT HOLDER),0.018524,0.107858,0.013146,0.709677
172,(RED HANGING HEART T-LIGHT HOLDER),(WHITE HANGING HEART T-LIGHT HOLDER),0.035056,0.107858,0.023304,0.664773
674,"(JUMBO BAG PINK POLKADOT, JUMBO STORAGE BAG SU...",(JUMBO BAG RED RETROSPOT),0.011652,0.096504,0.010158,0.871795
691,"(JUMBO SHOPPER VINTAGE RED PAISLEY, JUMBO BAG ...",(JUMBO BAG RED RETROSPOT),0.015138,0.096504,0.013146,0.868421
678,"(JUMBO BAG WOODLAND ANIMALS, JUMBO SHOPPER VIN...",(JUMBO BAG RED RETROSPOT),0.012250,0.096504,0.010507,0.857724
...,...,...,...,...,...,...
95,(HERB MARKER BASIL),(HERB MARKER THYME),0.011553,0.011453,0.010109,0.875000
104,(HERB MARKER THYME),(HERB MARKER PARSLEY),0.011453,0.011353,0.010308,0.900000
102,(HERB MARKER ROSEMARY),(HERB MARKER PARSLEY),0.011503,0.011353,0.010308,0.896104
98,(HERB MARKER MINT),(HERB MARKER PARSLEY),0.011602,0.011353,0.010208,0.879828
